## Preprocessing

In [ ]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

pd.options.display.max_rows = 500
pd.options.display.max_columns = 50

In [ ]:
#  Import and read the charity_data.csv

df = pd.read_csv("Resources/charity_data.csv")
print(df.info())
df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(columns=["EIN", "NAME"])

In [ ]:
# Determine the number of unique values in each column.

for col in df.columns:
    print(f"{col} \n{df[col].value_counts()}\n\n")

In [ ]:
def reduce_cats(a_col, a_cutoff):
    """ Inputs are a series and a cutoff value for 'Other' """
    
    print(f"BEFORE: \n\n{df[a_col].value_counts()}\n\n")

    types_to_replace = (df[a_col].value_counts().loc[lambda x: x < int(a_cutoff)]).keys().tolist()

    for code in types_to_replace:        
        df[a_col] = df[a_col].replace(code, "Other")

    # Check to make sure binning was successful

    print(f"AFTER: \n\n{df[a_col].value_counts()}\n\n")   

In [ ]:
reduce_cats("APPLICATION_TYPE", 27_000)

reduce_cats("AFFILIATION", 18_000)

reduce_cats("CLASSIFICATION", 17_000)

reduce_cats("USE_CASE", 28_000)

reduce_cats("ORGANIZATION", 23_000)

reduce_cats("ASK_AMT", 25_000)

In [ ]:
df

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`

alphasoup_cat = df.dtypes[df.dtypes == "object"].index.tolist() 
alphasoup_cat

In [ ]:
df[alphasoup_cat].nunique() 

In [ ]:
df1 = pd.get_dummies(df["APPLICATION_TYPE"], prefix="APP_TYPE", prefix_sep='_')
df2 = pd.get_dummies(df["AFFILIATION"], prefix="AFFIL", prefix_sep='_')
df3 = pd.get_dummies(df["CLASSIFICATION"], prefix="CLASS", prefix_sep='_')
df4 = pd.get_dummies(df["USE_CASE"], prefix="USE_CASE", prefix_sep='_')
df5 = pd.get_dummies(df["ORGANIZATION"], prefix="ORG", prefix_sep='_')
df6 = pd.get_dummies(df["INCOME_AMT"], prefix="INCOME", prefix_sep='_')
df7 = pd.get_dummies(df["SPECIAL_CONSIDERATIONS"], prefix="SPECIAL_CONSID", prefix_sep='_')
df8 = pd.get_dummies(df["ASK_AMT"], prefix="ASK_AMT", prefix_sep='_')
df9 = pd.get_dummies(df["STATUS"], prefix="STATUS", prefix_sep='_')

In [ ]:
enc_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df["IS_SUCCESSFUL"]], axis=1)
enc_df.info()

In [ ]:
enc_df

In [ ]:
# Split our preprocessed data into our features and target arrays

y = enc_df["IS_SUCCESSFUL"].values
X = enc_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0) 

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

print(X_train_scaled.shape)
print(X_test_scaled.shape)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=75, input_dim=25, activation="relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=75, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [ ]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=10) 

In [ ]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file

nn.save("AlphabetSoupCharity.h5")